In [1]:
from flask import Flask, request, jsonify, render_template
import pickle
import numpy as np
import pandas as pd
import string
import regex as re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

app = Flask(__name__)
model_a = pickle.load(open("model/a_classifier.pkl", "rb"))
model_sa = pickle.load(open("model/sa_classifier.pkl", "rb"))
model_ss = pickle.load(open("model/ss_classifier.pkl", "rb"))
tfidf_a = pickle.load(open("model/a_vectorizer.pkl", "rb"))
tfidf_sa = pickle.load(open("model/sa_vectorizer.pkl", "rb"))
tfidf_ss = pickle.load(open("model/ss_vectorizer.pkl", "rb"))

kamus = pd.read_csv("data/preprocessing/words.csv")
file_sw = open("data/preprocessing/stopwordbahasa.csv", "r")
kata_normalisasi_dict = {}
sw = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
remove_lists = ["nya", "pun", "yang", "an"]
exceptions_sw = ["tidak", "jangan", "guna", "manfaat"]
for index,row in kamus.iterrows():
    if row[0] not in kata_normalisasi_dict:
        kata_normalisasi_dict[row[0]] = row[1]
for line in file_sw:
    stripped_line = line.strip()
    sw.append(stripped_line)
for ex in exceptions_sw:
    if ex in sw:
        sw.remove(ex)
        
def vectorize(i_data,i_tfidf_vect_fit):
    X_tfidf = i_tfidf_vect_fit.transform(i_data)
    words = i_tfidf_vect_fit.get_feature_names()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)
def punc_casefold_lower_token(sentence):
    str_rm_punctuation = \
        sentence.translate(str.maketrans("", "", string.punctuation))
    str_no_number = re.sub('[^a-zA-Z]', ' ', str_rm_punctuation)
    return str_no_number.lower().split()
def pp_spelling(document):
    return [kata_normalisasi_dict[term] \
        if term in kata_normalisasi_dict \
            else term for term in document]
def pp_stemming(document):
    stem_list = []
    for word in document:
        word_stemmed = stemmer.stem(word)
        if word_stemmed in remove_lists:
            continue
        stem_list.append(word_stemmed)
    return stem_list
def pp_stopword(document):
    list_clean = []
    for word in document:
        if word in sw:
            continue
        list_clean.append(word)
    return list_clean
def pp_text(text):
    pp = punc_casefold_lower_token(text)
    pp = pp_spelling(pp)
    # pp = pp_stemming(pp)
    pp = pp_stopword(pp)
    return pp

In [31]:
text = ["Pelayanannya bagus, saya chat lewat whatsapp dibalas cepat"]
text = text[0]
input_text = text

text_pp = pp_text(text)

In [32]:
text_pp

['pelayanannya', 'bagus', 'memanggil', 'whatsapp', 'dibalas', 'cepat']

In [60]:
text_list = []
text_list.append(" ".join(text_pp))

tmp = []
tmp.append(text_list)
tmp
text_list = tmp
text_list

[['pelayanannya bagus memanggil whatsapp dibalas cepat']]

In [63]:
text_features = vectorize(text_list[0], tfidf_a)
text_features

,acak,ada,adanya,adil,admin,adminnya,aduan,aduannya,aduh,adukan,...,wilayah,wkwkwk,ya,yah,yak,yang,yasudahlah,ybs,yth,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
import numpy as np

nonzero = (text_features != 0).any()
a = nonzero.index[nonzero]
a = nonzero.index[nonzero].tolist()
a

dfnew = text_features[text_features.columns.intersection(a)]
dfnew

,bagus,cepat,dibalas,memanggil,whatsapp
0,0.28441,0.329154,0.583922,0.529061,0.435772


In [62]:
text_features["whatsapp"]

0    0.435772
Name: whatsapp, dtype: float64

In [75]:
prediction_a = model_a.predict(text_features)

In [77]:
prediction_a[0]

'service'

In [78]:
output_a = prediction_a[0]
prediction_s = ""
text_features = vectorize(text_list[0], tfidf_ss)
prediction_s = model_ss.predict(text_features)

In [80]:
nonzero = (text_features != 0).any()
a = nonzero.index[nonzero]
a = nonzero.index[nonzero].tolist()
a

dfnew = text_features[text_features.columns.intersection(a)]
dfnew

,bagus,cepat,dibalas,memanggil,whatsapp
0,0.324965,0.310019,0.576636,0.541663,0.415184


In [79]:
prediction_s[0]

'positive'